location txt folder to csv

In [ ]:
import os
import pandas as pd
from math import sqrt
import numpy as np

def process_files(directory):
    # 初始化一个空的DataFrame来收集所有数据
    all_data = pd.DataFrame(columns=['x channel', 'ymin', 'ymax', 'vmin', 'vmax', 'slope', 'length', 'class'])

    # 遍历指定目录下的所有文件
    for filename in os.listdir(directory):
        if filename.startswith("12CO_") and filename.endswith("_fila.txt"):
            x_channel = int(filename.split('_')[1][1:])  # 提取x channel的值
            file_path = os.path.join(directory, filename)
            with open(file_path, 'r') as file:
                next(file)  # 跳过第一行
                for line in file:
                    ymin, ymax, vmin, vmax = map(int, line.split())
                    # 计算斜率和几何长度
                    if (ymax - ymin) != 0:
                        slope = round((vmax - vmin) / (ymax - ymin), 4)
                    else:
                        slope = 0
                    length = round(sqrt((ymax - ymin)**2 + (vmax - vmin)**2), 4)
                    # 二分分类
                    binary_class = 1 if length >= 20 else 0
                    # 添加数据到DataFrame
                    all_data = all_data.append({
                        'x channel': x_channel,
                        'ymin': ymin,
                        'ymax': ymax,
                        'vmin': vmin,
                        'vmax': vmax,
                        'slope': slope,
                        'length': length,
                        'class': binary_class
                    }, ignore_index=True)

    # 排序
    all_data = all_data.sort_values(by=['x channel', 'ymin'])

    # 计算总结构数
    total_structures = len(all_data)

    # 添加 total 结构数列
    total_column_name = f'total: {total_structures}'  # 创建带总数的列名
    all_data[total_column_name] = np.nan  # 为所有行添加空值

    # 保存DataFrame到CSV文件
    output_path = os.path.join(directory, 'location-20.csv')  # 使用directory变量定义输出路径
    all_data.to_csv(output_path, index=False)

# 使用示例
directory = '/Users/naoj306/Desktop/find/aquila_12/location/nonan_diff_1-99_3-3'

process_files(directory)


add hms degree to csv

In [ ]:
from astropy.io import fits
from astropy.wcs import WCS
import pandas as pd
import os

def add_astronomical_coords_and_velocity(csv_path, fits_path, output_csv_path):
    with fits.open(fits_path) as hdul:
        header = hdul[0].header
        wcs = WCS(header)

    df = pd.read_csv(csv_path)

    # 遍历CSV中的每一行
    for index, row in df.iterrows():
        x = row['x channel']
        ymin = row['ymin']
        ymax = row['ymax']
        vmin = row['vmin']
        vmax = row['vmax']

        # 获取WCS转换结果
        result_ymin = wcs.pixel_to_world(x, ymin, vmin)
        result_ymax = wcs.pixel_to_world(x, ymax, vmax)

        # 检查返回值类型并提取SkyCoord对象
        if isinstance(result_ymin, WCS):
            sky_coord_ymin = result_ymin
            sky_coord_ymax = result_ymax
        else:
            sky_coord_ymin = result_ymin[0]
            sky_coord_ymax = result_ymax[0]

        # 存储转换后的坐标和速度值
        df.at[index, 'x(hms)'] = sky_coord_ymin.ra.to_string(unit='hour', sep=':')
        df.at[index, 'x(degree)'] = sky_coord_ymin.ra.deg
        df.at[index, 'ymin(hms)'] = sky_coord_ymin.dec.to_string(unit='degree', sep=':')
        df.at[index, 'ymin(degree)'] = sky_coord_ymin.dec.deg
        df.at[index, 'ymax(hms)'] = sky_coord_ymax.dec.to_string(unit='degree', sep=':')
        df.at[index, 'ymax(degree)'] = sky_coord_ymax.dec.deg
        # df.at[index, 'vmin(km/s)'] = (sky_coord_ymin.radial_velocity.to('m/s') / 1000.0).value if sky_coord_ymin.radial_velocity else None
        # df.at[index, 'vmax(km/s)'] = (sky_coord_ymax.radial_velocity.to('m/s') / 1000.0).value if sky_coord_ymax.radial_velocity else None

    output_full_path = os.path.join(output_csv_path, 'location-30_wcs.csv')
    if not os.path.exists(output_csv_path):
        os.makedirs(output_csv_path)
    
    df.to_csv(output_full_path, index=False)

csv_path = '/Users/naoj306/Desktop/find/aquila_12/location/nonan_diff_1-99_3-3/location-30.csv'
fits_path = '/Users/naoj306/Desktop/DB/SFP/AquilaRift_12CO_21.7arcsec_vel0.1_sph_v1.0.fits'
output_csv_path = '/Users/naoj306/Desktop/find/aquila_12/location/nonan_diff_1-99_3-3/'
add_astronomical_coords_and_velocity(csv_path, fits_path, output_csv_path)
